# Ноутбук с примером работы для грузов с синтетическими позициями (из графа OSM)
Чтобы не спамить гис, в качестве сервиса поиска расстояний используется граф OSM

In [1]:
import sys
sys.path.append('../')

In [2]:
import datetime
import random

import folium
import pandas as pd
from itertools import groupby
import pickle
import networkx as nx
import osmnx as ox
import os
from scripts.cargo_delivery import delivery, PathBuildingResult
from scripts.data_model import Cargo, Tariff, TariffCost
from itertools import count

from heapq import heappush, heappop

Загрузка и валидация данных (грузы только между двумя точками)

In [3]:
df_mos = pd.read_excel('../data/05_Грузы (только 2 точки) 2025-04-07.xlsx')
df_cpy = df_mos

In [4]:
df_cpy['Кол-во заявок по ручному ID (со схожей датой сбора и городами доставки и отправления)'].unique()

array([969, 394,  41,  13,   7,  35,  17, 282, 129,   1, 416, 264, 656,
       191, 165, 121,  84, 138,  26, 108,  66, 139,  24, 104, 128,  94,
        83,   6, 115,  76, 106,  79,   2,  11,   4, 181,  10, 120,  87,
       117,  52,  30, 136,  50,  48,  90,  44,   3,  42,  43, 589, 177,
        45,   8,  14,   5,  12,  37,  15,   9,  54, 118,  67,  39,  23,
        89,  36,  29,  81,  18,  25, 176, 111,  20,  22,  33,  19,  21,
        16,  61,  74,  28,  32, 273,  47,  34, 281, 285, 233,  27,  92,
        56, 182, 100,  57,  40,  31,  49,  60])

In [5]:
df_mos = df_cpy[df_cpy['Кол-во заявок по ручному ID (со схожей датой сбора и городами доставки и отправления)'] == 108]
df_mos = df_mos[df_mos['Количество точек'] == 2]

Вытаскиваем характеристики по массе объему по разным машинам

In [6]:
cars_df = pd.read_excel('../data/Автомобили Логистика.xlsx', sheet_name='Лист2')
cars_df = cars_df[cars_df['Категория груза'] == 'Обычные']
car_name_to_mass_volume = {r['Наименование автомобиля']: (r['Грузоподъемность, (кг)'], r['Объем, (м³)']) for _, r in
                           cars_df.iterrows() if
                           'миксер' not in r['Наименование автомобиля'] and r['Грузоподъемность, (кг)'] <= 10000}
del cars_df

In [7]:
car_name_to_mass_volume

{'ТС 7,0 тонн с гидролифтом': (7000, 33.0),
 'ТС 0,75 тонн': (750, 1.713408),
 'Средний грузовик': (3000, 23.0),
 'Большой грузовик': (5000, 36.0),
 'ТС 7.0 тонн': (7000, 44.0),
 'Мерседес Крытый 7 тонн': (7000, 44.0),
 'Фургон': (1500, 12.0),
 'Малый грузовик': (1500, 16.0),
 'Газель 1.5т': (1500, 12.0),
 'ТС 1,5 тонн': (1500, 8.0),
 'ТС 3,0 тонн': (3000, 18.5),
 'ТС 5,0 тонн': (5000, 35.0),
 'ТС 5,0 тонн с гидролифтом': (5000, 35.0),
 'ТС 3,0 тонн с гидролифтом': (3000, 18.5),
 'ТС 0,9 тонн': (800, 2.0),
 'ТС 0,05': (50, 0.5),
 'ТС 0,45 тонн': (450, 1.0),
 'ТС 10,0 тонн с гидролифтом': (10000, 50.0)}

Вытасуиваем тарифы

In [8]:
cost_df = pd.read_excel('../data/Тарифы Логистика.xlsx')
cost_df = cost_df[cost_df.apply(lambda r: 'Москва' in r['Регион'] and 'и' not in r['Регион'], axis=1)]

In [9]:
tariffs: list[Tariff] = []

for _, r in cost_df.iterrows():
    if r['Автомобиль'] not in car_name_to_mass_volume:
        continue
    tariffs.append(
        Tariff(
            id=r['Автомобиль'],
            mass=car_name_to_mass_volume[r['Автомобиль']][0],
            volume=car_name_to_mass_volume[r['Автомобиль']][1],
            cost_per_distance=[
                TariffCost(
                    min_dst_km=0,
                    max_dst_km=r['Минимальная поездка, км'],
                    cost_per_km=0,
                    fixed_cost=int(r['Стоимость мин. поездки, коп'] / 100)
                ),
                TariffCost(
                    min_dst_km=r['Минимальная поездка, км'],
                    max_dst_km=r['Максимальная протяженность маршрута, км'],
                    cost_per_km=int(r['Стоимость за км'] / 100),
                    fixed_cost=0
                )
            ])
    )

In [10]:
del cost_df

In [11]:
tariffs

[Tariff(id='ТС 0,75 тонн', mass=750, volume=1.713408, cost_per_distance=[TariffCost(min_dst_km=0, max_dst_km=30.0, cost_per_km=0, fixed_cost=990), TariffCost(min_dst_km=30.0, max_dst_km=2000.0, cost_per_km=33, fixed_cost=0)]),
 Tariff(id='ТС 3,0 тонн', mass=3000, volume=18.5, cost_per_distance=[TariffCost(min_dst_km=0, max_dst_km=30.0, cost_per_km=0, fixed_cost=1171), TariffCost(min_dst_km=30.0, max_dst_km=800.0, cost_per_km=39, fixed_cost=0)]),
 Tariff(id='ТС 3,0 тонн', mass=3000, volume=18.5, cost_per_distance=[TariffCost(min_dst_km=0, max_dst_km=30.0, cost_per_km=0, fixed_cost=1171), TariffCost(min_dst_km=30.0, max_dst_km=800.0, cost_per_km=39, fixed_cost=0)]),
 Tariff(id='ТС 3,0 тонн с гидролифтом', mass=3000, volume=18.5, cost_per_distance=[TariffCost(min_dst_km=0, max_dst_km=30.0, cost_per_km=0, fixed_cost=1574), TariffCost(min_dst_km=30.0, max_dst_km=800.0, cost_per_km=52, fixed_cost=0)]),
 Tariff(id='ТС 5,0 тонн с гидролифтом', mass=5000, volume=35.0, cost_per_distance=[TariffC

Удаление дублей тарифов (опционально, просто в таблице много лишних дублей)

In [12]:
def key(t: Tariff):
    return t.mass, t.volume


new_tariffs = []
for k, v in groupby(sorted(tariffs, key=key), key=key):
    v = list(v)[0]
    new_tariffs.append(v)
tariffs = new_tariffs

In [13]:
len(tariffs)

5

# Загрузка графа OSM

In [14]:
def get_path(folder: str, name: str):
    if not os.path.exists('../data'):
        os.mkdir('../data')
    path = os.path.join('../data', folder)
    if not os.path.exists(path):
        os.mkdir(path)
    return os.path.join(path, name)


# load graph
def get_graph(city_id: str = 'R2555133', dir=False) -> nx.Graph | nx.DiGraph:
    id_graph = city_id
    name = f'{id_graph}.pickle' if not dir else f'{id_graph}_dit.pickle'
    path = get_path('graphs', name)
    if os.path.exists(path):
        if dir:
            with open(path, 'rb') as fp:
                g: nx.DiGraph = pickle.load(fp)
                fp.close()
        else:
            with open(path, 'rb') as fp:
                g: nx.Graph = pickle.load(fp)
                fp.close()
    else:
        g = _get_gr(id_graph, dir)
        with open(path, 'wb') as fp:
            pickle.dump(g, fp)
            fp.close()
    assert g is not None
    g.remove_edges_from(nx.selfloop_edges(g))
    return g


def _get_gr(city_id, dir):
    gdf = ox.geocode_to_gdf(city_id, by_osmid=True)
    polygon_boundary = gdf.unary_union
    graph = ox.graph_from_polygon(polygon_boundary,
                                  network_type='drive',
                                  simplify=True)
    if dir:
        H = nx.DiGraph()
    else:
        H = nx.Graph()
    # Добавляем рёбра в новый граф, копируя только веса
    for u, d in graph.nodes(data=True):
        H.add_node(u, x=d['y'], y=d['x'])
    for u, v, d in graph.edges(data=True):
        if u == v:
            continue
        H.add_edge(u, v, length=d['length'])
    del city_id, gdf, polygon_boundary, graph
    return H



Это граф, на котором будут считаться расстояния, чтобы не спамить лищний раз гис

In [15]:
GRAPH_ID = 'R2555133'  # R13470549 R2555133 R3766483
# примеры id есть в graph_osm_loader.py
g = get_graph(GRAPH_ID)
print(len(g.nodes), len(g.edges))

18071 27261


семл рандомных "адресов"

In [16]:
addresses = random.sample(list(g.nodes), 100)

In [17]:
cargos: list[Cargo] = []


def get_time(param):
    param = str(param)
    if len(param) > 10:
        if '.' in param:
            return datetime.datetime.strptime(param, '%d.%m.%Y %H:%M:%S')
        else:
            return datetime.datetime.strptime(param, '%Y-%m-%d %H:%M:%S')
    else:
        return datetime.datetime.strptime(param, '%Y-%m-%d')

# парсим грузы из датафрейма
for i, r in df_mos.iterrows():
    start_node = random.choice(addresses)  # берем рандомные адреса из списка
    end_node = random.choice(addresses)
    while start_node == end_node:
        end_node = random.choice(addresses)
    l = nx.dijkstra_path_length(g, start_node, end_node, weight='length')

    mass = min(r['Вес, кг'], 100000)
    volume = min(r['Объем, м3'], 10)
    cm = r['ТРАНСПОРТ Грузоподъемность, (кг)']
    cv = r['ТРАНСПОРТ Объем, (м³)']
    d = Cargo(
        id=r['Номер заявки'],
        nodes=[start_node, end_node],
        mass=[mass, -mass],
        volume=[volume, -volume],
        service_time_minutes=[15, 15]
    )
    cargos.append(d)

In [18]:
# метод поиска пути для osm нужем только для работы ноутбука
def find_path_to_set(
        graph: nx.Graph,
        start: int,
        ends: set[int],
        left_cost: float = 5000) -> dict[object, float]:
    """
        Поиск пути от точки до множества точек
    :param graph: 
    :param start: 
    :param ends: 
    :param left_cost: 
    :return: 
    """
    adjacency = graph._adj
    c = count()
    push = heappush
    pop = heappop
    dist = {start: (0.0, 0.0, None)}
    fringe = []
    push(fringe, (0.0, 0.0, next(c), start))
    visited = set()
    nodes = graph.nodes()

    while fringe:
        (d, d0, _, v) = pop(fringe)
        if v in ends:
            visited.add(v)
        if len(visited) == len(ends):
            break
        for u, e in adjacency[v].items():
            vu_dist = d0 + e['length']
            vu_dist_n = d + e['length']
            if u not in dist or dist[u][0] > vu_dist_n:
                dist[u] = (vu_dist_n, vu_dist, v)
                push(fringe, (vu_dist_n, vu_dist, next(c), u))

    return {u: d[1] for u, d in dist.items()}

In [19]:
from sklearn.neighbors import KDTree

node2number = {u: i for i, u in enumerate(g.nodes())}
number2node = {i: u for i, u in enumerate(g.nodes())}
tree = KDTree([[d['x'], d['y']] for u, d in g.nodes(data=True)], leaf_size=10)


# получение ближайшей ноды графа по координатам
def get_near_nodes(coords, k=1) -> list[int]:
    x, y = coords
    dst, res = tree.query([[x, y]], k=k)
    # tree.query_radius()
    return [number2node[u] for u in res[0]]

In [20]:
def get_distance_matrix(points, start2end, k_near=-1):
    """
        Построение матрицы смежности по графу из OSM
    """
    dsts = {}
    for p in points:
        if k_near <= 0:
            others = {p2 for p2 in points if p2 != p and (p2, p) not in dsts}
        elif k_near >= 1:
            others = {p2 for p2 in get_near_nodes((g.nodes()[p]['x'], g.nodes()[p]['y']), k=k_near) if
                      p2 != p and (p2, p) not in dsts}
        else:
            raise Exception
        if p in start2end:
            others.update(start2end[p])
        paths = find_path_to_set(g, p, others)
        for p2 in others:
            dsts[p, p2] = paths[p2]
            dsts[p2, p] = paths[p2]
    return dsts

In [21]:
coords = {u: [d['x'], d['y']] for u, d in g.nodes(data=True)}
start2end = {}
for crg in cargos:
    u, v = crg.nodes
    if u not in start2end:
        start2end[u] = set()
    if v not in start2end:
        start2end[v] = set()
    start2end[u].add(v)
    start2end[v].add(u)

dsts = get_distance_matrix(addresses, start2end, k_near=-1)
time_mat = {k:v/1000 for k,v in dsts.items()}

In [22]:
cargos = cargos[:10] 

In [23]:
tariffs[0]

Tariff(id='ТС 0,75 тонн', mass=750, volume=1.713408, cost_per_distance=[TariffCost(min_dst_km=0, max_dst_km=30.0, cost_per_km=0, fixed_cost=990), TariffCost(min_dst_km=30.0, max_dst_km=2000.0, cost_per_km=33, fixed_cost=0)])

# Основной запуск модели

In [24]:
result = delivery(
    cargos,
    tariffs,
    points_to_coordinate=coords,
    distance_matrix=dsts,
    time_matrix=time_mat
)

2025-05-15 11:17:25,855 - cargo_delivery [INFO] - начало мержа грузов
2025-05-15 11:17:25,856 - cargo_delivery [INFO] - грузы объединены: было 10 стало 10
2025-05-15 11:17:25,856 - cargo_delivery [INFO] - количество точек: 18
2025-05-15 11:17:25,856 - cargo_delivery [INFO] - Начало создания графа грузов
2025-05-15 11:17:25,857 - cargo_delivery [INFO] - Граф грузов создан: 20 нод и 380 ребер
2025-05-15 11:17:25,858 - routing_model [INFO] - problem size: 20
2025-05-15 11:17:25,858 - routing_model [INFO] - prepare data for model
2025-05-15 11:17:25,859 - routing_model [INFO] - Начало создания модели
2025-05-15 11:17:25,860 - routing_model [INFO] - Добавление размерности для расстояния
2025-05-15 11:17:25,861 - routing_model [INFO] - Добавление размерности для расстояния
2025-05-15 11:17:25,861 - routing_model [INFO] - Добавление стоимостей машин
2025-05-15 11:17:25,861 - routing_model [INFO] - Добавление ограничений для пройденного расстояния
2025-05-15 11:17:25,862 - routing_model [INFO]

In [25]:
result.simple_routes

[Route(id=0, path=[1045833212, 318100290, 9806916419, 442566951], tariff=Tariff(id='ТС 0,75 тонн', mass=750, volume=1.713408, cost_per_distance=[TariffCost(min_dst_km=0, max_dst_km=30.0, cost_per_km=0, fixed_cost=990), TariffCost(min_dst_km=30.0, max_dst_km=2000.0, cost_per_km=33, fixed_cost=0)])),
 Route(id=2, path=[340064686, 595756177], tariff=Tariff(id='ТС 0,75 тонн', mass=750, volume=1.713408, cost_per_distance=[TariffCost(min_dst_km=0, max_dst_km=30.0, cost_per_km=0, fixed_cost=990), TariffCost(min_dst_km=30.0, max_dst_km=2000.0, cost_per_km=33, fixed_cost=0)])),
 Route(id=3, path=[2857835098, 7776310268], tariff=Tariff(id='ТС 0,75 тонн', mass=750, volume=1.713408, cost_per_distance=[TariffCost(min_dst_km=0, max_dst_km=30.0, cost_per_km=0, fixed_cost=990), TariffCost(min_dst_km=30.0, max_dst_km=2000.0, cost_per_km=33, fixed_cost=0)])),
 Route(id=4, path=[4451788792, 11172738250], tariff=Tariff(id='ТС 0,75 тонн', mass=750, volume=1.713408, cost_per_distance=[TariffCost(min_dst_km=

In [26]:
result.cargo_to_route

{'OT-0001-00378028': 0,
 'OT-0001-00378066': 0,
 'OT-0001-00378161': 2,
 'OT-0001-00378047': 3,
 'OT-0001-00378142': 4,
 'OT-0001-00378104': 5,
 'OT-0001-00393324': 5,
 'OT-0001-00378123': 20,
 'OT-0013-00210047': 20,
 'OT-0001-00378085': 20}

In [27]:
len(cargos)

10

In [28]:
for route in result.routes:
    print('________________________________________')
    print(f"car: {route.id} mass:{route.tariff.mass} volume: {route.tariff.volume}")
    path = route.path
    g = result.cargo_graph
    max_mass = max(sum(g.nodes()[path[i]]['mass'] for i in range(j)) for j in range(len(path)))
    max_volume = max(sum(g.nodes()[path[i]]['volume'] for i in range(j)) for j in range(len(path)))
    print(f'mass: {max_mass}|{route.tariff.mass} == {max_mass / route.tariff.mass * 100:.2f}%')
    print(f'volume: {max_volume:.2f}|{route.tariff.volume:.2f} == {max_volume / route.tariff.volume * 100:.2f}%')

________________________________________
car: 0 mass:750 volume: 1.713408
mass: 40.0|750 == 5.33%
volume: 0.10|1.71 == 5.60%
________________________________________
car: 2 mass:750 volume: 1.713408
mass: 20.0|750 == 2.67%
volume: 0.05|1.71 == 2.80%
________________________________________
car: 3 mass:750 volume: 1.713408
mass: 20.0|750 == 2.67%
volume: 0.05|1.71 == 2.80%
________________________________________
car: 4 mass:750 volume: 1.713408
mass: 20.0|750 == 2.67%
volume: 0.05|1.71 == 2.80%
________________________________________
car: 5 mass:750 volume: 1.713408
mass: 50.0|750 == 6.67%
volume: 0.12|1.71 == 7.00%
________________________________________
car: 20 mass:1500 volume: 8.0
mass: 1040.0|1500 == 69.33%
volume: 0.46|8.00 == 5.70%


In [29]:
def draw_on_map(data: PathBuildingResult,
                weight: float = 3.5,
                ) -> folium.Map:
    _g = data.cargo_graph
    u = list(_g.nodes())[0]
    u_x, u_y = _g.nodes()[u]['x'], _g.nodes()[u]['y']
    m: folium.Map = folium.Map(
        location=[u_x, u_y],
        zoom_start=11,
        tiles="cartodb positron"
    )  # Координаты города
    coords = {
    }

    for i, route in enumerate(data.routes):
        points_group = folium.FeatureGroup(name=f"points_{i}_{route.tariff.id}", show=False)
        points_group.add_to(m)
        path = route.path
        for i in range(len(path)):
            u = path[i]
            du = _g.nodes()[u]
            x, y = du['x'] + random.random() / 1000, du['y'] + random.random() / 1000
            coords[i] = (x, y)
            time = sum(data.cargo_graph.edges()[path[j], path[j + 1]]['time'] + data.cargo_graph.nodes()[path[j]]['service_time'] for j in range(i))
            
            test = f"""
            
            номер груза:       {path[i][0]}<br>
            номер ноды в графе:       {path[i][1]}<br>
            порядок посещения:      {i}<br>
            время:                  {time:.2f}<br>
            время обслуживание:     {data.cargo_graph.nodes()[path[i]]['service_time']:.2f}<br>
            """
            popup = folium.Popup(test, max_width=300, min_width=300)
            color = _g.nodes()[path[i]]['color'] if 'color' in _g.nodes()[path[i]] else 'red' if _g.nodes()[path[i]][
                                                                                                     'mass'] < 0 else 'blue'
            folium.CircleMarker(
                location=(x, y),
                radius=8,
                fill=True,
                fill_color=color,
                color=color,
                fill_opacity=0.7,
                popup=popup
            ).add_to(points_group)

        for i in range(len(path) - 1):
            u_x, u_y = coords[i]
            v_x, v_y = coords[i + 1]
            mass_by_edge = sum(_g.nodes()[path[j]]['mass'] for j in range(i + 1))
            volume_by_edge = sum(_g.nodes()[path[j]]['volume'] for j in range(i + 1))

            test = f"""
                масса на плече: {mass_by_edge:.2f}<br>
                объем на плече: {volume_by_edge:.2f}<br>
                длина плеча:    {_g.edges()[path[i], path[i + 1]]['length'] / 1000:.2f} <br>
                время:          {_g.edges()[path[i], path[i + 1]]['time']:.2f}
            """
            popup = folium.Popup(test, max_width=300, min_width=300)
            folium.PolyLine(
                [(u_x, u_y), (v_x, v_y)],
                weight=weight,
                color='blue',
                popup=popup
            ).add_to(points_group)
    folium.LayerControl().add_to(m)
    return m

In [30]:
m = draw_on_map(result)

In [ ]:
m.show_in_browser()

Your map should have been opened in your browser automatically.
Press ctrl+c to return.


In [ ]:
# m.save('map.html')